In [31]:
#Dependencies
import pandas as pd
import numpy as np

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func
import MySQLdb
import mysql.connector

In [32]:
#Extract
housing_development = pd.read_csv('Assets/Data/Affordable_Rental_Housing_Developments.csv')
census_data = pd.read_csv('Assets/Data/Census_Data_-_Selected_socioeconomic_indicators_in_Chicago__2008___2012.csv')
vacant_building = pd.read_csv('Assets/Data/Vacant_and_Abandoned_Buildings_-_Violations.csv')

In [33]:
#clean/transform
housing_final = housing_development.dropna()
census_final = census_data.dropna()
vacant_final = vacant_building.dropna()

In [34]:
# vacant_building
census_data.tail()
#housing_development.tail()

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
73,74.0,Mount Greenwood,1.0,3.4,8.7,4.3,36.8,34381,16.0
74,75.0,Morgan Park,0.8,13.2,15.0,10.8,40.3,27149,30.0
75,76.0,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24.0
76,77.0,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0
77,NaN,CHICAGO,4.7,19.7,12.9,19.5,33.5,28202,NaN


In [35]:
census_data.at[77, 'Community Area Number']=0
#census_data

In [36]:
#Merging Census Data and housing development by Communit Area Number
Combined_data = pd.merge(housing_final, census_final, how='outer', on=['Community Area Number'])

In [37]:
# cleaning up ..
data = Combined_data.dropna()
data = data.drop(['COMMUNITY AREA NAME'], axis=1)

In [38]:
# connect to the database
database_path = "housing_data.db" #whatever you type in here will be the name of the database
engine = create_engine(f"sqlite:///{database_path}")
conn = engine.connect()

In [39]:
#Double check it worked
inspector = inspect(engine)
inspector.get_table_names()

[]

In [40]:
#insert the table into the db
data.to_sql(name='housing_data', con=engine, if_exists='replace', index=False)

In [41]:
#Double check it worked
inspector = inspect(engine)
inspector.get_table_names()

['housing_data']

In [42]:
database_path = "housing.db"
engine = create_engine(f"postgresql:///{database_path}")
conn = engine.connect()

OperationalError: (psycopg2.OperationalError) fe_sendauth: no password supplied
 (Background on this error at: http://sqlalche.me/e/e3q8)